In [1]:
#!/usr/bin/env python3
"""
Test runner for 3D Cellular Automaton simulation
"""

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Import the CA class (assuming ca_simulation_3d.py is in the same directory)
try:
    from settings_ca1 import CellularAutomatonRexGG3D
except ImportError:
    print("Error: Could not import CellularAutomatonRexGG3D")
    print("Make sure ca_simulation_3d.py is in the same directory")
    sys.exit(1)

def check_files():
    """Check if required files exist"""
    required_files = ['condition.yaml', '3grains.vti']
    missing_files = []
    
    for file in required_files:
        if not os.path.exists(file):
            missing_files.append(file)
    
    if missing_files:
        print("Error: Missing required files:")
        for file in missing_files:
            print(f"  - {file}")
        return False
    
    print("✓ All required files found")
    return True

def run_test_simulation():
    """Run the test simulation"""
    print("\n" + "="*60)
    print("3D Cellular Automaton Recrystallization Simulation")
    print("="*60 + "\n")
    
    # Check files
    if not check_files():
        return
    
    try:
        # Create CA instance
        print("Initializing simulation...")
        ca = CellularAutomatonRexGG3D('condition.yaml')
        
        # Print initial statistics
        stats = ca.get_statistics()
        print(f"\nInitial statistics:")
        print(f"  Grid size: {ca.nx} × {ca.ny} × {ca.nz} = {ca.n_cells} cells")
        print(f"  Cell size: {ca.dx*1e6:.1f} μm")
        print(f"  Number of grains: {stats['n_orientations']}")
        print(f"  Mean stored energy: {stats['mean_stored_energy']:.2e} J/m³")
        print(f"  Boundary cells: {stats['n_boundaries']}")
        
        # Run simulation
        print("\nRunning simulation...")
        print(f"  Max time: {ca.max_time} seconds")
        print(f"  Max steps: {ca.max_steps}")
        print(f"  Save interval: every {ca.save_interval} steps")
        print(f"  Output directory: {ca.output_dir}")
        print("\n" + "-"*60 + "\n")
        
        history = ca.run_simulation()
        
        # Print final statistics
        print("\n" + "-"*60)
        print("\nFinal statistics:")
        print(f"  Simulation time: {history['time'][-1]:.3f} seconds")
        print(f"  Total steps: {history['step'][-1]}")
        print(f"  Recrystallized fraction: {history['rex_fraction'][-1]:.3f}")
        print(f"  Final number of grains: {history['n_orientations'][-1]}")
        print(f"  Final mean stored energy: {history['mean_stored_energy'][-1]:.2e} J/m³")
        
        # Create summary plots
        create_summary_plots(ca, history)
        
        print(f"\n✓ Simulation completed successfully!")
        print(f"✓ Results saved in: {ca.output_dir}/")
        print(f"✓ VTI files: step_*.vti")
        print(f"✓ History data: simulation_history.npz")
        print(f"✓ Summary plots: simulation_summary.png")
        
    except Exception as e:
        print(f"\n✗ Error during simulation: {str(e)}")
        import traceback
        traceback.print_exc()

def create_summary_plots(ca, history):
    """Create enhanced summary plots"""
    fig = plt.figure(figsize=(15, 10))
    
    # Plot 1: Recrystallization kinetics
    ax1 = plt.subplot(2, 3, 1)
    ax1.plot(history['time'], history['rex_fraction'], 'b-', linewidth=2)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Recrystallized Fraction')
    ax1.set_title('Recrystallization Kinetics')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-0.05, 1.05)
    
    # Plot 2: JMAK plot
    ax2 = plt.subplot(2, 3, 2)
    # Calculate JMAK coordinates
    X = history['rex_fraction']
    X_valid = X[(X > 0.01) & (X < 0.99)]  # Avoid log(0) and log(inf)
    t_valid = history['time'][:len(X_valid)]
    if len(X_valid) > 2:
        jmak_y = np.log(-np.log(1 - X_valid))
        jmak_x = np.log(t_valid + 1e-6)  # Avoid log(0)
        ax2.plot(jmak_x, jmak_y, 'ro-', markersize=4)
        # Fit a line to get Avrami exponent
        if len(jmak_x) > 10:
            fit = np.polyfit(jmak_x[5:], jmak_y[5:], 1)
            ax2.plot(jmak_x, fit[0]*jmak_x + fit[1], 'k--', 
                    label=f'n = {fit[0]:.2f}')
            ax2.legend()
    ax2.set_xlabel('ln(time)')
    ax2.set_ylabel('ln(-ln(1-X))')
    ax2.set_title('JMAK (Avrami) Plot')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Grain count evolution
    ax3 = plt.subplot(2, 3, 3)
    ax3.plot(history['time'], history['n_orientations'], 'g-', linewidth=2)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Number of Grains')
    ax3.set_title('Grain Count Evolution')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Stored energy evolution
    ax4 = plt.subplot(2, 3, 4)
    ax4.semilogy(history['time'], history['mean_stored_energy'], 'm-', linewidth=2)
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Mean Stored Energy (J/m³)')
    ax4.set_title('Stored Energy Evolution')
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Simulation progress
    ax5 = plt.subplot(2, 3, 5)
    ax5.plot(history['step'], history['time'], 'c-', linewidth=2)
    ax5.set_xlabel('Simulation Step')
    ax5.set_ylabel('Time (s)')
    ax5.set_title('Time vs Steps')
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Rate plot
    ax6 = plt.subplot(2, 3, 6)
    if len(history['time']) > 1:
        dt = np.diff(history['time'])
        dX = np.diff(history['rex_fraction'])
        rate = dX / dt
        ax6.plot(history['time'][1:], rate, 'r-', linewidth=2)
        ax6.set_xlabel('Time (s)')
        ax6.set_ylabel('Recrystallization Rate (1/s)')
        ax6.set_title('Recrystallization Rate')
        ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(ca.output_dir, 'simulation_summary.png'), dpi=150)
    plt.close()

def check_output_files(output_dir):
    """Check and list output files"""
    if os.path.exists(output_dir):
        print(f"\nOutput files in {output_dir}:")
        vti_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.vti')])
        print(f"  - {len(vti_files)} VTI files")
        if len(vti_files) > 0:
            print(f"    First: {vti_files[0]}")
            print(f"    Last:  {vti_files[-1]}")
        
        other_files = [f for f in os.listdir(output_dir) if not f.endswith('.vti')]
        for f in other_files:
            print(f"  - {f}")

if __name__ == "__main__":
    # Run the test simulation
    run_test_simulation()
    
    # Check output files
    if os.path.exists('output_3grains_simulation'):
        check_output_files('output_3grains_simulation')
    
    print("\n" + "="*60)
    print("Simulation complete!")
    print("="*60)


3D Cellular Automaton Recrystallization Simulation

✓ All required files found
Initializing simulation...
Loading initial microstructure from: 3grains.vti

✗ Error during simulation: OrientationID array not found in VTI file

Output files in output_3grains_simulation:
  - 0 VTI files

Simulation complete!


Traceback (most recent call last):
  File "/var/folders/jk/f_qb8b2s5lqbc31h60wyn8gw0000gn/T/ipykernel_50793/2479442548.py", line 50, in run_test_simulation
    ca = CellularAutomatonRexGG3D('condition.yaml')
  File "/Users/namhoongoo/celluarauto/08_jul29_2025/settings_ca1.py", line 54, in __init__
    self._load_initial_microstructure()
  File "/Users/namhoongoo/celluarauto/08_jul29_2025/settings_ca1.py", line 94, in _load_initial_microstructure
    raise ValueError("OrientationID array not found in VTI file")
ValueError: OrientationID array not found in VTI file
